# DSCI 100  Project Final Report 

**Group 26** \
// TODO: put other names here \
Matt Wiens \#21158845

In [ ]:
import itertools
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
# Simplify working with large datasets in Altair
alt.data_transformers.enable("vegafusion")

# Output dataframes instead of arrays
set_config(transform_output="pandas")

In [ ]:
# Seed for sklearn random_state parameter
SEED = 1234

## Introduction

The Pacific Laboratory for Artificial Intelligence (PLAI) at UBC has provided data recorded from their Minecraft server, PLAICraft. The data were obtained by recording players’ gameplay and collecting user registration information, and include details about player activity and demographic attributes.

The project lead of PLAICraft, Frank Wood, is interested in answering the following question:

> "What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?"

This report will investigate that question. The general idea of what we will do is find features—either the variables directly included in the data or transformations of them—which help us predict player newsletter subscription status. 

### Data description

We'll first load in the data and then describe it.

In [ ]:
players = pd.read_csv("data/players.csv")
sessions = pd.read_csv("data/sessions.csv")

In [ ]:
players

In [ ]:
sessions

The `players` dataframe contains information about 196 players with the following 9 variables:

| Variable | Type | Description | Notes |
|---|---|---|---|
| experience | ordinal | Experience level  | Values have the following ordering:  Beginner, Amateur, Regular, Pro, Veteran |
| subscribe | boolean | Newsletter subscription status | &nbsp; |
| hashedEmail | string | Email (hashed) | Unique for each player and can be used to join the `sessions` dataframe (see below) |
| played_hours | numeric | Total gameplay hours | &nbsp; |
| name | string | First name | &nbsp; |
| gender | categorical | Gender | Gender includes more than two categories, e.g., Male, Female, Two-Spirited, Non-binary |
| age | numeric | Player's age | &nbsp; |
| individualId | unknown | Player's ID | No values included in the data |
| organizationName | unknown| Player's organization | No values included in the data |

The `sessions` dataframe contains information about 1535 gameplay sessions with the following 5 variables:

| Variable | Type | Description | Notes |
|---|---|---|---|
| hashedEmail | string | Player email (hashed) | Unique for each player and can be used to join the `players` dataframe |
| start_time | datetime (string) | Session start datetime | Stored as string in `DD/MM/YYYY HH:MM` format |
| end_time | datetime (string) | Session end datetime | Stored as string in `DD/MM/YYYY HH:MM` format |
| original_start_time | numeric | Session start datetime | Stored as Unix time in milliseconds |
| original_end_time | numeric | Session end datetime | Stored as Unix time in milliseconds |

## Methods and Results

### Overview

We want to identify which features help predict player newsletter subscription status. To do this, we will find the combination of features that allow a $k$-nearest neighbours (KNN) classifier to achieve the highest predictive accuracy.

First, we will perform exploratory data analysis to understand the data and identify candidate predictor features. Next, we will perform grid-search cross-validation using KNN, where the grid spans different combinations of predictors as well as the number of nearest neighbors used. The best-performing model from the grid will give us the combination of features that lead to the highest predictive accuracy.

###  Tidying and initial wrangling

The response variable we will predict is the `subscribe` variable from the `players` dataframe. The `players` dataframe gives us candidate predictor features `experience`, `played_hours`, `gender`, and `age`. We will also derive the following additional candidate predictor features from the `sessions` dataframe:

+ number of sessions
+ average session length

The below code obtains these features and combines them with the `players` columns. Important note: for players having no sessions, we fill in their average session length with 0, even though the true value is undefined.

In [ ]:
# Convert session start time and end time columnss to datetime objects
DATE_FORMAT = "%d/%m/%Y %H:%M"

sessions["start_time"] = pd.to_datetime(sessions["start_time"], format=DATE_FORMAT)
sessions["end_time"] = pd.to_datetime(sessions["end_time"], format=DATE_FORMAT)

# Get each session length in seconds
sessions["session_length"] = (
    sessions["end_time"] - sessions["start_time"]
).dt.total_seconds()

# Aggregate by hashed email
session_stats = (
    sessions.groupby("hashedEmail")
    .agg(
        num_sessions=("hashedEmail", "count"),
        avg_session_length_seconds=("session_length", "mean"),
    )
    .reset_index()
)

# Left join players with session stats on hashedEmail
players_combined_messy = players.merge(session_stats, on="hashedEmail", how="left")

# Fill in NaNs for players with 0 sessions
players_combined_messy["num_sessions"] = (
    players_combined_messy["num_sessions"].fillna(0).astype(int)
)
players_combined_messy["avg_session_length_seconds"] = players_combined_messy[
    "avg_session_length_seconds"
].fillna(0)


To tidy the data we'll drop the `individualId` and `organizationName` columns from the dataframe since they contain no values. Additionally, we'll perform the following transformations:

+ convert `experience` to an ordinal variable, which allows us to conveniently use their ordering
+ add a column `subscribe_label`, which labels the subcription status and will be convenient for plotting
+ drop columns we're not going to use: `hashedEmail` and `name`

In [ ]:
# Drop columns
players_combined = players_combined_messy.drop(
    ["individualId", "organizationName", "hashedEmail", "name"], axis=1
)

# Convert experience to an ordinal variable
players_combined["experience"] = pd.Categorical(
    players_combined["experience"],
    categories=["Beginner", "Amateur", "Regular", "Pro", "Veteran"],
    ordered=True,
)

# Add a column with label for subscription status
players_combined["subscribe_label"] = players_combined["subscribe"].map(
    {True: "Subscribed", False: "Not Subscribed"}
)

We'll do additional wrangling following our data exploration but for now our data is as follows:

In [ ]:
players_combined

### Splitting the data into training and test sets

Before exploring the data, we'll split the data into training and test sets using a 75%-25% stratified split based on the `subscribe` column.

In [ ]:
df_train, df_test = train_test_split(
    players_combined,
    test_size=0.25,
    stratify=players_combined["subscribe"],
    random_state=SEED,
)

### Data exploration

Now we'll explore our training data to obtain insights and identify candidate predictor features.

#### Overall subscription counts

Before diving into potential associations, let's get a baseline look at overall subscription counts.

In [ ]:
# Overall subscription counts
overall_counts_plot = (
    alt.Chart(df_train)
    .mark_bar()
    .encode(
        x=alt.X("subscribe_label").title("Subscription status").axis(labelAngle=0),
        y=alt.Y("count()").title("Player count"),
    )
    .properties(
        width=500,
        title=f"{df_train['subscribe'].mean():.0%} of Players Have Newsletter Subscriptions",
    )
)
overall_counts_plot

We can see that we have strongly imbalanced classes, which is an issue for KNN. There are different ways of addressing this issue, but we'll handle it by oversampling the minority class (done below in the "Minority class oversampling" section).

This also gives us the majority-class baseline. If our classifer scores less than 73% accuracy, it performs worse than always predicting the majority class.

#### Subscription status by experience level

We'll first look at how subscription status associates with experience level.

In [ ]:
# Proportion subscribed vs experience level
proportion_subscribed_by_experience = (
    df_train.groupby("experience", observed=True)["subscribe"]
    .agg([("count", "size"), ("proportion_subscribed", "mean")])
    .reset_index()
)
proportion_subscribed_by_experience

In [ ]:
experience_plot = (
    alt.Chart(proportion_subscribed_by_experience)
    .mark_bar()
    .encode(
        x=alt.X("experience").title("Experience level").axis(labelAngle=0),
        y=alt.Y("proportion_subscribed")
        .title("Percentage subscribed to newsletter")
        .axis(format="%")
        .scale(domain=[0, 1]),
    )
    .properties(
        width=500,
        title="Proportion of Players Subscribed by Experience Level",
    )
)
experience_plot

Experience level is very weakly associated with subscription status. We won't consider it as a predictor feature.

#### Subscription status by gender

Next we'll look at subscription status and gender.

In [ ]:
# Proportion subscribed vs gender
proportion_subscribed_by_gender = (
    df_train.groupby("gender")["subscribe"]
    .agg([("count", "size"), ("proportion_subscribed", "mean")])
    .reset_index()
)
proportion_subscribed_by_gender

We can see some of the genders have low counts—the proportion subscribed isn't very meaningful in these cases, so we'll exclude these genders in the plot below.

In [ ]:
gender_plot = (
    alt.Chart(
        proportion_subscribed_by_gender.loc[
            proportion_subscribed_by_gender["count"] > 5
        ]
    )
    .mark_bar()
    .encode(
        x=alt.X("gender").title("Gender").axis(labelAngle=0).sort("-y"),
        y=alt.Y("proportion_subscribed")
        .title("Percentage subscribed to newsletter")
        .axis(format="%")
        .scale(domain=[0, 1]),
    )
    .properties(
        width=500,
        title="Proportion of Players Subscribed by Gender (excluding genders with ≤5 players)",
    )
)
gender_plot

Players who choose to declare their gender seem to be much more likely to subscribe than players who do not declare their gender. This gives us a good predictor feature: whether a player has declared their gender.

#### Subscription status by gameplay hours

Now we'll look at how subscription status associates with gameplay hours. Ideally, we would compare the distribution of gameplay hours by subscription status using side-by-side boxplots. Unfortunately, as shown below, the vast majority of players have negligible gameplay hours, which will collapse any boxplot into a line.

In [ ]:
# Histogram of hours played
hours_hist_plot = (
    alt.Chart(df_train)
    .mark_bar()
    .encode(
        x=alt.X("played_hours").bin(step=2).title("Gameplay hours"),
        y=alt.Y("count()").title("Player count"),
    )
    .properties(
        width=500,
        title="Distribution of Gameplay Hours",
    )
)
hours_hist_plot

To address this, we'll break the data into two groups of players: one having less than one hour of gameplay, the other having at least one hour.

For players with less than one hour of gameplay, we'll use a bar chart of subscription counts, as their gameplay hours are too concentrated for boxplots to be informative. For players with at least one hour, we'll compare gameplay hours by subscription status using boxplots.

In [ ]:
# Subscription counts for players with <1 hour of gameplay
low_hours_counts_plot = (
    alt.Chart(df_train.loc[df_train["played_hours"] < 1])
    .mark_bar()
    .encode(
        x=alt.X("subscribe_label").title("Subscription status").axis(labelAngle=0),
        y=alt.Y("count()").title("Player count"),
    )
    .properties(
        width=500,
        title=f"{df_train.loc[df_train['played_hours'] < 1]['subscribe'].mean():.0%} of Players with Less Than One Hour of Gameplay Have Newsletter Subscriptions",
    )
)
low_hours_counts_plot

Comparing this with our baseline subscription counts, there appears to be essentially no association between very low gameplay hours and subscription status.

In [ ]:
# Gameplay hours vs subscription status (at least 1 hour played)
high_hours_plot = (
    alt.Chart(df_train.loc[df_train["played_hours"] >= 1])
    .mark_boxplot(size=60)
    .encode(
        x=alt.X("subscribe_label").title("Subscription status").axis(labelAngle=0),
        y=alt.Y("played_hours").title("Gameplay hours"),
    )
    .properties(
        width=500,
        title="Distribution of Played Hours by Subscription Status (at least 1 hour gameplay)",
    )
)
high_hours_plot

Here we can clearly see that having at least an hour of gameplay is strongly associated with subscription status. For players with a large number of hours, all of them are subscribed.

To use gameplay hours as a predictor we have a choice to keep it continuous or transform it into a binary "less than one hour played"/"at least one hour played" variable. If there are associations within narrower gameplay hours ranges for players having less than one hour played, then by using the binary variable we are losing this information. However, if those associations don't exist, using the continuous variable just adds noise to the data. In lieu of more exhaustive data analysis, we'll take the conservative approach and take gameplay hours as a continuous predictor feature.

#### Subscription status by number of gameplay sessions

When looking at how number of gameplay sessions associates with subscription status, we're going to run into the same issue we had for gameplay hours: the majority of players will have very few gameplay sessions. We'll approach this issue the same way and look at players with less than 2 sessions separately from players with at least 2 sessions.

In [ ]:
# Subscription counts for players with <2 sessions
low_num_sessions_counts_plot = (
    alt.Chart(df_train.loc[df_train["num_sessions"] < 2])
    .mark_bar()
    .encode(
        x=alt.X("subscribe_label").title("Subscription status").axis(labelAngle=0),
        y=alt.Y("count()").title("Player count"),
    )
    .properties(
        width=500,
        title=f"{df_train.loc[df_train['num_sessions'] < 2]['subscribe'].mean():.0%} of Players with Less Than Two Gameplay Sessions Have Newsletter Subscriptions",
    )
)
low_num_sessions_counts_plot

Similarly to gameplay hours, we see that there is very little association between very few gameplay sessions and newsletter subscription status.

In [ ]:
# Number of sessions vs subscription status (at least 2 sessions)
high_num_sessions_plot = (
    alt.Chart(df_train.loc[df_train["num_sessions"] >= 2])
    .mark_boxplot(size=60)
    .encode(
        x=alt.X("subscribe_label").title("Subscription status").axis(labelAngle=0),
        y=alt.Y("num_sessions").title("Number of Sessions"),
    )
    .properties(
        width=500,
        title="Distribution of Number of Sessions by Subscription Status (at least 2 sessions)",
    )
)
high_num_sessions_plot

Here we again see a similarity with gameplay hours, where having a significant number of gameplay hours is strongly associated with gameplay hours.

Gameplay hours and number of sessions associate very similarly with subscription status. It's worth considering whether it's redundant to have both of them as predictor features. To explore this, we first look at their Pearson correlation coefficient:

In [ ]:
float(df_train["played_hours"].corr(df_train["num_sessions"]))

We can see that while the variables are strongly linearly related, there is enough independent variation to justify including both. We therefore include number of sessions as a predictor feature (as a continuous variable, by the same arguments made in the last section).

#### Subscription status by average session length

Now we'll at how average session length associates with subscription status. Recall that when we calculated this variable we filled in undefined average session lengths (for players with no sessions) with 0. We'll look at positive values here.

In [ ]:
# Average session length vs subscription status
avg_session_length_plot = (
    alt.Chart(df_train[df_train["avg_session_length_seconds"] > 0])
    .transform_calculate(
        avg_session_length_minutes="datum.avg_session_length_seconds / 60"
    )
    .mark_boxplot(size=60)
    .encode(
        x=alt.X("subscribe_label").title("Subscription status").axis(labelAngle=0),
        y=alt.Y("avg_session_length_minutes:Q").title(
            "Average Session Length (minutes)"
        ),
    )
    .properties(
        width=500, title="Distribution of Average Session Length by Subscription Status"
    )
)
avg_session_length_plot

We can see a moderate strength association between average session length and subscription status, so we'll include this as a predictor feature.

#### Subscription status by age

Finally, we will look at how age is associated with subscription status.

In [ ]:
# Age vs subscription status
age_plot = (
    alt.Chart(df_train)
    .mark_boxplot(size=60)
    .encode(
        x=alt.X("subscribe_label").title("Subscription status").axis(labelAngle=0),
        y=alt.Y("age").title("Age"),
    )
    .properties(width=500, title="Distribution of Age by Subscription Status")
)
age_plot

Depending on the seed used, it may be hard to see the median for the subscribed distribution. To clarify this, we show that the median is close to (or equal to) the first quartile:

In [ ]:
df_train[df_train["subscribe"]]["age"].describe()

We can see that if a player is subscribed, they will tend to be younger, although because of the large overlap in the distributions, this effect is weak. We'd argue it's still strong enough to use as a predictor feature, however.

### Summarizing the predictor features we'll consider

The predictor features we'll partition into different combinations during training are as follows:

+ gender disclosure status, which we'll need to one-hot encode
+ gameplay hours
+ number of sessions
+ average session length
+ age

We'll add in the gender disclosure status feature to both training and test data now.

In [ ]:
def add_gender_disclosure(df: pd.DataFrame) -> pd.DataFrame:
    return df.assign(
        gender_disclosure_status=np.where(
            df["gender"] == "Prefer not to say", "unspecified", "specified"
        )
    )

In [ ]:
df_train_final = add_gender_disclosure(df_train)
df_test_final = add_gender_disclosure(df_test)

### Minority class oversampling

Next we'll fix the class imbalance by oversampling the non-subscribed data.

In [ ]:
n_extra = 2 * df_train_final["subscribe"].sum() - df_train_final.shape[0]

df_train_final_resampled = pd.concat(
    [
        df_train_final,
        df_train_final[~df_train_final["subscribe"]].sample(
            n=n_extra, replace=True, random_state=SEED
        ),
    ]
)

Side note 1: the number of additional observations $N_{\text{extra}}$ we need for the minority class must satisfy
$$N_{\text{majority}} = N_{\text{minority}} + N_{\text{extra}}.$$

Substituting
$$N_{\text{minority}} = N_{\text{total}} - N_{\text{majority}}$$
and rearranging, we get
$$N_{\text{extra}} = 2 N_{\text{majority}} - N_{\text{total}}.$$

Side note 2: We could also address the class imbalance by weighing the majority vote by inverse distance. However, combining oversampling with inverse-distance weighing risks overfitting the data given the class imbalance is large and the dataset is small, as the duplicated points could become extremely influential. We therefore choose only one of these methods.

### Training

We're now ready to start training with grid-search cross-validation. We'll first partition our data into predictor features and response variable.

In [ ]:
def partition_data(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.Series]:
    return (
        df[
            [
                "gender_disclosure_status",
                "played_hours",
                "num_sessions",
                "avg_session_length_seconds",
                "age",
            ]
        ],
        df["subscribe"],
    )

In [ ]:
X_train, y_train = partition_data(df_train_final_resampled)
X_test, y_test = partition_data(df_test_final)

#### Setting up the grid

To have search over different combinations of features, we'll need to make a column selector to use in the pipeline.

In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns]

Next we'll define the grid values—the different feature combinations—that the column selector consumes.

In [ ]:
features = [
    "gender_disclosure_status",
    "played_hours",
    "num_sessions",
    "avg_session_length_seconds",
    "age",
]
feature_combinations: list[list[str]] = []

# NOTE: we skip single feature classifiers, which are typically very weak
for r in range(2, len(features) + 1):
    feature_combinations.extend(map(list, itertools.combinations(features, r=r)))

We'll also define the number of nearest neighbour grid values.

In [ ]:
k_vals = [3, 5, 7, 9, 11]

#### Grid-search cross-validation

Next we'll set up the pipeline for the grid-search, and perform 5-fold grid-search cross-validation.

In [ ]:
preprocessor = make_column_transformer(
    (
        StandardScaler(),
        ["played_hours", "num_sessions", "avg_session_length_seconds", "age"],
    ),
    (
        OneHotEncoder(
            drop="if_binary",
            sparse_output=False,
            feature_name_combiner=lambda feature, category: feature,  # keep column name
        ),
        ["gender_disclosure_status"],
    ),
    verbose_feature_names_out=False,
)

# NOTE: we don't use make_pipeline here because we want to explicitly name each
# stage
# NOTE: we preprocess first before selecting columns even though it's a bit
# computationally wasteful. This avoids headaches with missing features that the
# preprocessor expects. This is fine for our tiny dataset, but maybe not for
# larger training data.
pipeline = Pipeline(
    [
        ("preprocess", preprocessor),
        ("select", ColumnSelector()),
        ("knn", KNeighborsClassifier()),
    ]
)

param_grid = {
    "select__columns": feature_combinations,
    "knn__n_neighbors": k_vals,
}
# NOTE: this should run fast (a few seconds on my machine), but if it doesn't
# uncomment the "verbose=3" line to get more output to see what's going on
grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    # verbose=3,
)

grid.fit(X_train, y_train);

### Results

TODO: finish this!

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

accuracy_score(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

In [ ]:
# just a test...
mask = X_test["num_sessions"] > 1
X_test_2 = X_test[mask]
y_pred_2 = best_model.predict(X_test_2)

accuracy_score(y_pred_2, y_test[mask])

In [ ]:
y_majority = pd.Series(True, index=y_test[mask].index)
accuracy_score(y_pred_2, y_majority)

## Discussion